# Tools in Data Science - Project 1

In [2]:
# Imports
import pandas as pd
import statsmodels.api as sm
from collections import Counter
from datetime import datetime

# Load data
users_df = pd.read_csv('users.csv')
repositories_df = pd.read_csv('repositories.csv')

In [ ]:
# 1. Who are the top 5 users in Zurich with the highest number of followers? List their login in order, comma-separated.

# Sort by followers and get top 5
top_users = users_df.sort_values(by='followers', ascending=False).head(5)

# Extract logins
top_logins = top_users['login'].tolist()
result = ','.join(top_logins)

print(result)

IDouble,TheOfficialFloW,Seldaek,riscv,JonnyBurger


In [8]:
# 2. Who are the 5 earliest registered GitHub users in Zurich? List their login in ascending order of created_at, comma-separated.

# Convert created_at to datetime
users_df['created_at'] = pd.to_datetime(users_df['created_at'])

# Sort by created_at and get the earliest 5 users
earliest_users = users_df.sort_values(by='created_at').head(5)

# Extract logins
earliest_logins = earliest_users['login'].tolist()
result = ','.join(earliest_logins)

print(result)

lejoe,uwolfer,matthiask,oscardelben,panterch


In [9]:
# 3. What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.

# Filter out missing license names
repositories_df = repositories_df[repositories_df['license_name'].notna()]

# Count occurrences of each license
license_counts = repositories_df['license_name'].value_counts()

# Get the top 3 licenses
top_licenses = license_counts.head(3).index.tolist()

# Join the license names in order
result = ','.join(top_licenses)

print(result)

mit,other,apache-2.0


In [10]:
# 4. Which company do the majority of these developers work at?

# Count occurrences of each company
company_counts = users_df['company'].value_counts()

# Get the company with the highest count
most_common_company = company_counts.idxmax()
most_common_count = company_counts.max()

print(f"The majority of developers work at: {most_common_company} with {most_common_count} developers.")

The majority of developers work at: GOOGLE with 51 developers.


In [11]:
# 5. Which programming language is most popular among these users?

# Count occurrences of each programming language, ignoring missing values
language_counts = repositories_df['language'].value_counts()

# Get the most popular programming language
most_popular_language = language_counts.idxmax()
most_popular_count = language_counts.max()

print(f"The most popular programming language is: {most_popular_language} with {most_popular_count} repositories.")

The most popular programming language is: Python with 2558 repositories.


In [12]:
# 6. Which programming language is the second most popular among users who joined after 2020?

# Convert created_at to datetime and filter users who joined after 2020
users_df['created_at'] = pd.to_datetime(users_df['created_at'])
recent_users = users_df[users_df['created_at'] > '2020-01-01']

# Get the logins of recent users
recent_user_logins = recent_users['login'].tolist()

# Filter repositories by these users
recent_repositories = repositories_df[repositories_df['login'].isin(recent_user_logins)]

# Count occurrences of each programming language
language_counts = recent_repositories['language'].value_counts()

# Get the second most popular programming language
second_most_popular_language = language_counts.nlargest(2).index[1]
second_most_popular_count = language_counts.nlargest(2).values[1]

print(f"The second most popular programming language among users who joined after 2020 is: {second_most_popular_language} with {second_most_popular_count} repositories.")

The second most popular programming language among users who joined after 2020 is: JavaScript with 27 repositories.


In [13]:
# 7. Which language has the highest average number of stars per repository?

# Group by programming language and calculate the average stars
average_stars = repositories_df.groupby('language')['stargazers_count'].mean()

# Identify the language with the highest average stars
highest_average_language = average_stars.idxmax()
highest_average_value = average_stars.max()

print(f"The programming language with the highest average number of stars per repository is: {highest_average_language} with an average of {highest_average_value:.2f} stars.")

The programming language with the highest average number of stars per repository is: BitBake with an average of 364.00 stars.


In [14]:
# 8. Let's define leader_strength as followers / (1 + following). Who are the top 5 in terms of leader_strength? List their login in order, comma-separated.

# Calculate leader_strength
users_df['leader_strength'] = users_df['followers'] / (1 + users_df['following'])

# Sort by leader_strength and get the top 5
top_leaders = users_df.sort_values(by='leader_strength', ascending=False).head(5)

# Extract logins
top_logins = top_leaders['login'].tolist()
result = ','.join(top_logins)

print(result)

riscv,bpasero,Seldaek,egamma,ethz-asl


In [15]:
# 9. What is the correlation between the number of followers and the number of public repositories among users in Zurich?

# Calculate the correlation between followers and public repositories
correlation = users_df['followers'].corr(users_df['public_repos'])

print(f"The correlation between the number of followers and the number of public repositories is: {correlation:.3f}")

The correlation between the number of followers and the number of public repositories is: 0.065


In [16]:
# 10. Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.

# Define the independent variable (X) and dependent variable (Y)
X = users_df['public_repos']
Y = users_df['followers']

# Add a constant to the independent variable (for the intercept)
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(Y, X).fit()

# Get the summary of the regression results
summary = model.summary()

# Extract the coefficient for public_repos
additional_followers_per_repo = model.params['public_repos']

print(f"Estimated additional followers per additional public repository: {additional_followers_per_repo:.3f}")

Estimated additional followers per additional public repository: 1.477


In [6]:
# 11. Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?

# Calculate the correlation directly
correlation = repositories_df['has_projects'].astype(int).corr(repositories_df['has_wiki'].astype(int))

print(f"The correlation between having projects enabled and having a wiki enabled is: {correlation:.3f}")


The correlation between having projects enabled and having a wiki enabled is: 0.312


In [18]:
# 12. Do hireable users follow more people than those who are not hireable?

# Filter hireable and non-hireable users
hireable_users = users_df[users_df['hireable'] == True]
non_hireable_users = users_df[users_df['hireable'].isna() | (users_df['hireable'] == False)]

# Calculate average following for both groups
average_hireable_following = hireable_users['following'].mean()
average_non_hireable_following = non_hireable_users['following'].mean()

# Calculate the difference
difference = average_hireable_following - average_non_hireable_following

# Print the result rounded to three decimal places
print(f'Difference in average following (hireable - non-hireable): {difference:.3f}')

Difference in average following (hireable - non-hireable): -844.119


In [20]:
# 13. Some developers write long bios. Does that help them get more followers? What's the impact of the length of their bio (in Unicode words, split by whitespace) with followers? (Ignore people without bios)

# Filter out users without bios
users_with_bios = users_df[users_df['bio'].notna()].copy()

# Calculate the word count of each bio
users_with_bios['bio_word_count'] = users_with_bios['bio'].apply(lambda x: len(x.split()))

# Prepare the data for regression
X = users_with_bios['bio_word_count']  # Independent variable
y = users_with_bios['followers']        # Dependent variable

# Add a constant to the independent variable for the regression
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Get the regression slope (coefficient for bio_word_count)
slope = model.params['bio_word_count']

# Print the slope rounded to three decimal places
print(f'Regression slope of followers on bio word count: {slope:.3f}')


Regression slope of followers on bio word count: 40.894


In [21]:
# 14. Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

# Convert the created_at column to datetime
repositories_df['created_at'] = pd.to_datetime(repositories_df['created_at'], errors='coerce')

# Drop rows with invalid dates
repositories_df = repositories_df.dropna(subset=['created_at'])

# Filter for weekend days (Saturday: 5, Sunday: 6)
weekend_repos = repositories_df[repositories_df['created_at'].dt.dayofweek.isin([5, 6])]

# Count the number of repositories created by each user on weekends
weekend_repo_counts = Counter(weekend_repos['login'])

# Get the top 5 users who created the most repositories on weekends
top_users = weekend_repo_counts.most_common(5)

# Extract the logins of the top users
top_logins = [user[0] for user in top_users]

# Output the top users' logins as a comma-separated string
print(','.join(top_logins))

mrcjkb,JonnyBurger,eregon,kynan,lris


In [3]:
# 15. Do people who are hireable share their email addresses more often?

# Total number of users
total_users = len(users_df)

# Filter hireable and non-hireable users
hireable_users = users_df[users_df['hireable'] == True]
non_hireable_users = users_df[users_df['hireable'].isna() | (users_df['hireable'] == False)]

# Calculate the fraction of users with email in both groups
fraction_hireable_with_email = hireable_users['email'].notna().mean()
fraction_non_hireable_with_email = non_hireable_users['email'].notna().mean()

# Calculate the difference
difference = fraction_hireable_with_email - fraction_non_hireable_with_email

# Print the result rounded to three decimal places
print(f'Difference in fraction of users with email: {difference:.3f}')

Difference in fraction of users with email: 0.068


In [5]:
# 16. Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)

# Filter out users without names
valid_users = users_df[users_df['name'].notna()].copy()

# Extract surnames (last word in name)
valid_users['surname'] = valid_users['name'].str.strip().str.split().str[-1]

# Count occurrences of each surname
surname_counts = valid_users['surname'].value_counts()

# Find the most common surname(s)
max_count = surname_counts.max()
most_common_surnames = surname_counts[surname_counts == max_count].index.tolist()

# Sort surnames alphabetically
most_common_surnames.sort()

# Count users with the most common surname
number_of_users = max_count

# Print results
most_common_surnames_str = ','.join(most_common_surnames)
print(f'Most common surname(s): {most_common_surnames_str}')

Most common surname(s): Li,Wang
